# Analyzing Well Bundles

## 1. Importing / Installing Packages

In [1]:
import pandas as pd # Importing pandas package

# Set the maximum number of columns to display to None
pd.set_option('display.max_columns', None)

import numpy as np # Importing numpy package

from typing import Dict, Tuple, List, Union # Importing specific types from typing module

import re # Importing regular expression package

from src.database_manager import DatabricksOdbcConnector # Importing DatabricksOdbcConnector class from database_manager module
from src.utils import reorder_columns # Importing reorder_columns function from utils module

from scipy.spatial.distance import cdist # Importing cdist function from scipy package

import time

import pyproj # Importing pyproj package

## 2. Loading Excel/csv into Pandas DataFrame

In [2]:
df_raw = pd.read_csv('wellHeader_with_Cluster.csv',dtype={'ChosenID':str},parse_dates=['FirstProdDate','Comp_Dt'])

In [3]:
# Renaming cluster column to bundle
df_raw = df_raw.rename(columns={'cluster':'bundle'})

In [4]:
# Sort by a specific column (e.g., 'bundle') in ascending order
df_raw.sort_values(by='bundle', ascending=True, ignore_index=True, inplace=True)

In [5]:
df_raw.shape

(2514, 20)

## 3. Data Preprocessing

### 3.1. Creating DSU Columns

In [6]:
# Creating DSU columns names from Lease Name columns

df_raw['DSU'] = df_raw['LeaseName'].apply(
    lambda x: re.sub(r'[^a-zA-Z\s]', ' ',  # Remove special characters, keep letters and spaces
                     re.match(r'([^\d]+)', str(x)).group(1) if pd.notna(x) and re.match(r'([^\d]+)', str(x)) else str(x))  
                    .strip()  # Strip leading/trailing spaces
).replace(r'\s+', ' ', regex=True)  # Collapse multiple spaces into a single space

# Placing DSU next to LeaseName
df_raw = reorder_columns(df=df_raw, columns_to_move=['DSU'], reference_column='LeaseName')

## 4. Feature Engineering

### 4.1. Creating dataframes that have more than one unique bundles or DSU

In [7]:
# Filter rows where the same DSU has more than one unique bundle
same_DSU_diffBundle_df = df_raw[df_raw.groupby("DSU")["bundle"].transform("nunique") > 1]

In [8]:
# Filter rows where the same bundle has more than one unique DSU
same_Bundle_diffDSU_df = df_raw[df_raw.groupby("bundle")["DSU"].transform("nunique") > 1]

### 4.2. Defining Functions that is used in calculation for i-k pair dataframe

In [9]:
def extract_heel_toe_mid_lat_lon(well_trajectory: pd.DataFrame) -> pd.DataFrame:
    """
    Extract the heel, toe, and mid-point latitude/longitude for each ChosenID in the well trajectory DataFrame.

    Parameters:
    well_trajectory: pd.DataFrame
        DataFrame containing well trajectory data, including 'ChosenID', 'md', 'latitude', and 'longitude'.

    Returns:
    pd.DataFrame
        A DataFrame with 'ChosenID', 'Heel_Lat', 'Heel_Lon', 'Toe_Lat', 'Toe_Lon', 'Mid_Lat', 'Mid_Lon'.

    Example:
    >>> data = {
    ...     "ChosenID": [1001, 1001, 1001, 1002, 1002],
    ...     "md": [5000, 5100, 5200, 6000, 6100],
    ...     "latitude": [31.388, 31.389, 31.387, 31.400, 31.401],
    ...     "longitude": [-103.314, -103.315, -103.316, -103.318, -103.319]
    ... }
    >>> df = pd.DataFrame(data)
    >>> extract_heel_toe_mid_lat_lon(df)
       ChosenID  Heel_Lat  Heel_Lon  Toe_Lat  Toe_Lon  Mid_Lat  Mid_Lon
    0     1001    31.388  -103.314   31.387  -103.316  31.3875 -103.315
    1     1002    31.400  -103.318   31.401  -103.319  31.4005 -103.3185
    """
    # Ensure the data is sorted by MD in ascending order
    well_trajectory = well_trajectory.sort_values(by=["ChosenID", "md"], ascending=True)

    # Group by 'ChosenID' and extract heel/toe lat/lon
    heel_toe_df = (
        well_trajectory.groupby("ChosenID")
        .agg(
            heel_lat=("latitude", "first"),
            heel_lon=("longitude", "first"),
            toe_lat=("latitude", "last"),
            toe_lon=("longitude", "last"),
        )
        .reset_index()
    )

    # Calculate midpoints
    heel_toe_df["mid_Lat"] = (heel_toe_df["heel_lat"] + heel_toe_df["toe_lat"]) / 2
    heel_toe_df["mid_Lon"] = (heel_toe_df["heel_lon"] + heel_toe_df["toe_lon"]) / 2

    return heel_toe_df

In [10]:
def get_direction(lat1: np.ndarray, lon1: np.ndarray, lat2: np.ndarray, lon2: np.ndarray) -> np.ndarray:
    """
    Determine the relative direction of (lat2, lon2) with respect to (lat1, lon1).
    
    Parameters:
    lat1, lon1: np.ndarray
        Latitude and longitude of the first well.
    lat2, lon2: np.ndarray
        Latitude and longitude of the second well.
    
    Returns:
    np.ndarray
        Array indicating the direction (e.g., North, South, East, West) of well B relative to well A.
    """
    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1

    conditions = [
        np.abs(lat_diff) > np.abs(lon_diff),
        lat_diff > 0,
        lon_diff > 0
    ]

    choices = ["North", "South", "East", "West"]
    
    return np.select(
        [conditions[0] & conditions[1], conditions[0] & ~conditions[1], ~conditions[0] & conditions[2], ~conditions[0] & ~conditions[2]],
        choices
    )

In [11]:
def calculate_drill_direction_vectorized(well_trajectories: Dict[str, pd.DataFrame], i_indices: np.ndarray) -> np.ndarray:
    """
    Vectorized function to determine the drilling direction of multiple wells using NumPy operations.
    
    Parameters:
    well_trajectories: Dict[str, pd.DataFrame]
        Dictionary containing well trajectory data indexed by ChosenID.
    i_indices: np.ndarray
        Array of ChosenID whose drill directions need to be calculated.
    
    Returns:
    np.ndarray
        Array containing "EW" (East-West) or "NS" (North-South) for each well.
    """
    azimuth_values = np.array([well_trajectories[i]["azimuth"].mean() if not well_trajectories[i].empty else np.nan for i in i_indices])
    
    conditions = (45 <= azimuth_values) & (azimuth_values < 135) | (225 <= azimuth_values) & (azimuth_values < 315)
    drill_directions = np.where(np.isnan(azimuth_values), "Unknown", np.where(conditions, "EW", "NS"))
    
    return drill_directions

In [12]:
def optimized_calculate_3D_distance_matrix(
    trajectories: Dict[str, pd.DataFrame], i_indices: np.ndarray, k_indices: np.ndarray
) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Fully vectorized 3D distance calculations for well pairs using NumPy and Pandas.
    
    Parameters:
    trajectories: Dict[str, pd.DataFrame]
        Dictionary containing well trajectory data indexed by well ID.
    i_indices: np.ndarray
        Array of well IDs representing the first well in each pair.
    k_indices: np.ndarray
        Array of well IDs representing the second well in each pair.
    
    Returns:
    Tuple[np.ndarray, np.ndarray, np.ndarray]
        - Horizontal distances between the well pairs.
        - Vertical distances between the well pairs.
        - 3D distances between the well pairs.
    """
    # 🚀 Precompute mean (midpoint) for each well ID across all wells at once
    all_trajectories_df = pd.concat(trajectories.values(), keys=trajectories.keys()).reset_index(drop=True)

    midpoints_df = all_trajectories_df.groupby("ChosenID")[["x", "y", "tvd"]].mean()

    # Convert to NumPy arrays for fast lookup
    well_ids = midpoints_df.index.to_numpy()
    midpoints = midpoints_df.to_numpy()

    # Create a mapping from well ID to its index
    well_id_to_idx = {well_id: idx for idx, well_id in enumerate(well_ids)}

    # Efficiently extract midpoints using NumPy indexing
    mid_A = midpoints[np.array([well_id_to_idx[i] for i in i_indices])]
    mid_B = midpoints[np.array([well_id_to_idx[k] for k in k_indices])]

    # Compute distances
    vertical_distances = np.abs(mid_A[:, 2] - mid_B[:, 2])
    mid_B[:, 2] = mid_A[:, 2]  # Align Well B to Well A’s TVD

    horizontal_distances = np.linalg.norm(mid_A[:, :2] - mid_B[:, :2], axis=1)
    total_3D_distances = np.sqrt(horizontal_distances**2 + vertical_distances**2)

    return horizontal_distances, vertical_distances, total_3D_distances

In [13]:
def create_i_k_pairs(df: pd.DataFrame, trajectories: Union[Dict[str, pd.DataFrame], pd.DataFrame]) -> pd.DataFrame:
    """
    Generate the i_k_pairs DataFrame, computing horizontal and vertical distances, 
    3D distances, drilling directions, and relative directions between well pairs.
    
    Parameters:
    df: pd.DataFrame
        DataFrame containing well metadata with:
        - "ChosenID" (str): Unique well identifier.

    trajectories: Union[Dict[str, pd.DataFrame], pd.DataFrame]
        Either:
        - A dictionary mapping well IDs ("ChosenID") to trajectory DataFrames.
        - A single DataFrame containing all trajectory data (must have "ChosenID" column).
        
    Each trajectory DataFrame should include:
    - "md" (float): Measured depth.
    - "tvd" (float): True vertical depth.
    - "inclination" (float): Inclination angle in degrees.
    - "azimuth" (float): represents the drilling direction.
    - "latitude" (float): Latitude values, define the geographical position.
    - "longitude" (float): Longitude values, define the geographical position.
    - "x" (float): X-coordinate in a Cartesian coordinate system.
    - "y" (float): Y-coordinate in a Cartesian coordinate system.
    
    Returns:
    pd.DataFrame
        DataFrame containing pairs of wells (`i_uwi`, `k_uwi`) with their computed distances 
        and directional relationships.
    """
    start_time = time.time()

    # Convert to dictionary if input is a DataFrame
    if isinstance(trajectories, pd.DataFrame):
        if "ChosenID" not in trajectories.columns:
            raise ValueError("🚨 Error: Trajectory DataFrame must contain a 'ChosenID' column.")
        trajectories = {cid: group for cid, group in trajectories.groupby("ChosenID")}

    step1_time = time.time()
    print(f"✅ Step 1: Trajectory DataFrame conversion took {step1_time - start_time:.4f} seconds.")

    # Get unique ChosenIDs from df and validate existence in trajectories
    chosen_ids = df["ChosenID"].unique()
    missing_ids = [cid for cid in chosen_ids if cid not in trajectories]

    if missing_ids:
        print(f"⚠️ The following ChosenIDs do not exist in the trajectory data and will be excluded: {missing_ids}")

    df = df[df["ChosenID"].isin(trajectories)]
    
    step2_time = time.time()
    print(f"✅ Step 2: ChosenID validation and filtering took {step2_time - step1_time:.4f} seconds.")

    # Generate i-k pair indices
    n = len(df)
    i_idx, k_idx = np.triu_indices(n, k=1)

    i_uwi = df.iloc[i_idx]["ChosenID"].values
    k_uwi = df.iloc[k_idx]["ChosenID"].values

    step3_time = time.time()
    print(f"✅ Step 3: i-k pair generation took {step3_time - step2_time:.4f} seconds.")

    # 🚀 Optimized Heel/Toe Extraction (Vectorized)
    heel_toe_df = pd.concat(
        [extract_heel_toe_mid_lat_lon(trajectories[cid]) for cid in df['ChosenID'].unique()], ignore_index=True
    )
    heel_toe_dict = heel_toe_df.set_index("ChosenID").to_dict(orient="index")

    step4_time = time.time()
    print(f"✅ Step 4 (Optimized): Heel/Toe extraction took {step4_time - step3_time:.4f} seconds.")

    # Efficiently extract values using vectorized lookups
    heel_lat_i = np.array([heel_toe_dict[i]["heel_lat"] for i in i_uwi])
    heel_lon_i = np.array([heel_toe_dict[i]["heel_lon"] for i in i_uwi])
    toe_lat_k = np.array([heel_toe_dict[k]["toe_lat"] for k in k_uwi])
    toe_lon_k = np.array([heel_toe_dict[k]["toe_lon"] for k in k_uwi])

    step5_time = time.time()
    print(f"✅ Step 5: Heel/Toe dictionary lookup took {step5_time - step4_time:.4f} seconds.")

    # 🚀 Optimized Distance Calculation (Fully Vectorized)
    horizontal_dist, vertical_dist, total_3D_dist = optimized_calculate_3D_distance_matrix(trajectories, i_uwi, k_uwi)

    step6_time = time.time()
    print(f"✅ Step 6 (Optimized): Distance calculations took {step6_time - step5_time:.4f} seconds.")

    # Compute drill directions
    drill_directions = calculate_drill_direction_vectorized(trajectories, i_uwi)

    step7_time = time.time()
    print(f"✅ Step 7: Drill direction calculation took {step7_time - step6_time:.4f} seconds.")

    # Determine directional relationship
    ward_of_i = get_direction(heel_lat_i, heel_lon_i, toe_lat_k, toe_lon_k)

    step8_time = time.time()
    print(f"✅ Step 8: Directional relationship calculation took {step8_time - step7_time:.4f} seconds.")

    # Create DataFrame
    result_df = pd.DataFrame({
        "i_uwi": i_uwi,
        "k_uwi": k_uwi,
        "horizontal_dist": horizontal_dist,
        "vertical_dist": vertical_dist,
        "3D_ft_to_same": total_3D_dist,
        "drill_direction": drill_directions,
        "ward_of_i": ward_of_i
    })

    total_time = time.time()
    print(f"🚀 Total Execution Time: {total_time - start_time:.4f} seconds.")

    return result_df

In [14]:
def calculate_overlap(well_A: pd.DataFrame, well_B: pd.DataFrame) -> float:
    """
    Calculate the percentage overlap between two horizontal wellbores.
    
    Parameters:
    well_A: pd.DataFrame
        Well trajectory data for Well A, including 'MD' (Measured Depth) and 'Inclination'.
    well_B: pd.DataFrame
        Well trajectory data for Well B, including 'MD' (Measured Depth) and 'Inclination'.
    
    Returns:
    float:
        Percentage of overlap relative to the shorter lateral.
    """
    if well_A.empty or well_B.empty:
        return 0.0

    start_A, end_A = well_A["MD"].min(), well_A["MD"].max()
    start_B, end_B = well_B["MD"].min(), well_B["MD"].max()

    overlap_start = max(start_A, start_B)
    overlap_end = min(end_A, end_B)

    if overlap_start >= overlap_end:
        return 0.0

    overlap_length = overlap_end - overlap_start
    shorter_length = min(end_A - start_A, end_B - start_B)

    return (overlap_length / shorter_length) * 100 if shorter_length > 0 else 0.0

### 4.2. Defining Functions that is used to compute Lat/Lon to UTM Co-Ordinates

In [15]:
def determine_utm_zone(longitude: float) -> int:
    """
    Determines the UTM zone based on a given longitude.
    """
    return int((longitude + 180) / 6) + 1


def batch_latlon_to_utm(lat: np.ndarray, lon: np.ndarray, utm_zone: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    Converts arrays of latitudes and longitudes to UTM coordinates in meters for a given UTM zone.
    """
    proj_utm = pyproj.Transformer.from_crs(
        "EPSG:4326", f"EPSG:326{utm_zone}", always_xy=True
    )
    
    return proj_utm.transform(lon, lat)


def compute_utm_coordinates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Computes UTM (x, y, z) coordinates for multiple wells, using surface location to determine UTM zones.
    Converts UTM coordinates from meters to feet. Uses vectorized batch processing for performance.

    Parameters:
    - df (pd.DataFrame): Original directional survey DataFrame.

    Returns:
    - pd.DataFrame: DataFrame with all original columns + x, y, z (in feet), and utm_zone.
    """
    start_time = time.time()  # Start timing

    # Step 1: Sort dataframe by md to identify surface location
    df = df.sort_values(by=["ChosenID", "md"], ascending=[True, True])
    
    # Step 2: Determine UTM zones using the surface location (first row per well)
    surface_locs = df.groupby("ChosenID").first()[["latitude", "longitude"]]
    surface_locs["utm_zone"] = surface_locs["longitude"].apply(determine_utm_zone)

    # Merge UTM zones back into the original dataframe
    df = df.merge(surface_locs[["utm_zone"]], on="ChosenID", how="left")

    print(f"✅ Determined UTM zones in {time.time() - start_time:.4f} seconds.")

    # Step 3: Batch transformation for each unique UTM zone
    start_transform_time = time.time()
    unique_zones = df["utm_zone"].unique()
    utm_converters: Dict[int, Tuple[np.ndarray, np.ndarray]] = {}

    for zone in unique_zones:
        subset = df[df["utm_zone"] == zone]
        easting, northing = batch_latlon_to_utm(subset["latitude"].values, subset["longitude"].values, zone)
        utm_converters[zone] = (easting, northing)

    print(f"✅ Performed batch EPSG transformations in {time.time() - start_transform_time:.4f} seconds.")

    # Step 4: Assign the converted coordinates back to the DataFrame
    start_assign_time = time.time()
    df["x"], df["y"] = np.zeros(len(df)), np.zeros(len(df))

    for zone in unique_zones:
        mask = df["utm_zone"] == zone
        df.loc[mask, "x"], df.loc[mask, "y"] = utm_converters[zone]

    print(f"✅ Assigned transformed coordinates in {time.time() - start_assign_time:.4f} seconds.")

    # Step 5: Convert UTM coordinates from meters to feet (Conversion factor: 1 meter = 3.28084 feet)
    df["x"] *= 3.28084
    df["y"] *= 3.28084
    
    df["z"] = -df["tvd"] # Elevation is negative TVD

    print(f"✅ Total execution time: {time.time() - start_time:.4f} seconds.")

    return df

## 5. Testinig

In [16]:
# Importing Directional Survey data from Databricks

databricks = DatabricksOdbcConnector()

# Filtering only Horizontal wells and getting their apis
chosen_ids = ", ".join(f"'{id}'" for id in df_raw[df_raw['HoleDirection']=='H']['ChosenID'].unique())

try:
    databricks.connect()

    query = f"""
    SELECT
        LEFT(uwi, 10) AS ChosenID, 
        station_md_uscust AS md, 
        station_tvd_uscust AS tvd,
        inclination, 
        azimuth, 
        latitude, 
        longitude, 
        x_offset_uscust AS `deviation_E/W`,
        ew_direction,
        y_offset_uscust AS `deviation_N/S`,
        ns_direction
        
    FROM ihs_sp.well.well_directional_survey_station
    WHERE LEFT(uwi, 10) IN ({chosen_ids})
    ORDER BY uwi, md;
    """

    df_directional = databricks.execute_query(query)

except Exception as e:
    print(f"Error: {e}")

finally:
    databricks.close_connection()

c:\users\apoorva.saxena\onedrive - sitio royalties\desktop\project - apoorva\python\parent_child_spacing\src\database_manager.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result_df = pd.read_sql(sql_query, self.connection)


In [17]:
df_with_utm = compute_utm_coordinates(df_directional)

✅ Determined UTM zones in 0.6978 seconds.
✅ Performed batch EPSG transformations in 0.2321 seconds.
✅ Assigned transformed coordinates in 0.0158 seconds.
✅ Total execution time: 0.9622 seconds.


In [18]:
df_ik_pairs = create_i_k_pairs(df=df_raw, trajectories=df_with_utm[df_with_utm['inclination']>=89][['ChosenID','md','tvd','inclination','azimuth','latitude','longitude','x','y']])

✅ Step 1: Trajectory DataFrame conversion took 0.0999 seconds.
⚠️ The following ChosenIDs do not exist in the trajectory data and will be excluded: ['4238935759', '4238935783', '4238935872', '4238933169', '4238940091', '4238937586', '4238910461', '4238934764', '4238930501', '4238932256', '4238932191', '4238932194', '4238910497', '4238937713', '4238933558', '4238930262', '4238933989', '4238931100', '4238937707', '4238910420', '4238939527', '4238940159', '4238940321', '4238940597', '4238940596', '4238940594', '4238940625']
✅ Step 2: ChosenID validation and filtering took 0.0020 seconds.
✅ Step 3: i-k pair generation took 0.6069 seconds.
✅ Step 4 (Optimized): Heel/Toe extraction took 16.2367 seconds.
✅ Step 5: Heel/Toe dictionary lookup took 4.9940 seconds.
✅ Step 6 (Optimized): Distance calculations took 4.4364 seconds.
✅ Step 7: Drill direction calculation took 149.6608 seconds.
✅ Step 8: Directional relationship calculation took 0.1172 seconds.
🚀 Total Execution Time: 177.0338 seconds.

In [ ]:
# df_ik_pairs.to_csv('i_k_pairs_Reeves_v1.csv',index=False)